This notebook is a proto for the ETL process which will be complete in etl.py

In [ ]:
import psycopg2
import os
import glob
import pandas as pd
from sql_queries import * 

In [ ]:
try: 
    conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=postgres password=admin")
except psycopg2.Error as e: 
    print("Error: Could not make connection to the Postgres database")
    print(e)
try: 
    cur = conn.cursor()
except psycopg2.Error as e: 
    print("Error: Could not get cursor to the Database")
    print(e)
conn.set_session(autocommit=True)

In [ ]:
# the following funtion lists all files in a given filepath
def get_files(filepath):
    all_files=[]
    for root, dirs, files in os.walk(filepath):
        files=glob.glob(os.path.join(root,'*.json'))
        for f in files:
            all_files.append(os.path.abspath(f))
    return all_files

### Process song_data to extract songs & artists dimensional tables
- In this notebook, for the sake of demo, I'll perform ETL on a single song file and load a single record into each table to start.
- I'll use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- I'll select the first song in that list
- I'll read the song file and view the data

In [ ]:
song_files=get_files("data/song_data")

In [ ]:
filepath=song_files[0]

In [ ]:
df_songs=pd.read_json(filepath, lines=True)

In [ ]:
df_songs

#### Songs Table 

Select the columns needed to populate songs table

In [ ]:
songs_df=df_songs[["song_id", "title", "artist_id", "year", "duration"]].values[0]

In [ ]:
songs_data=songs_df.tolist()

In [ ]:
songs_data

Implement the `song_table_insert` in `sql_queries.py` to insert the above record into `songs` table.
Remember to run create_tables.py before running the cell below to ensure you have created/resetted the `songs` table in `sparkifydb` database

In [ ]:
cur.execute(song_table_insert, songs_data)

#### Artists table

Select the columns needed to populate artists table

In [ ]:
artists_df=df_songs[["artist_id", "artist_name", "artist_location", "artist_latitude", "artist_longitude"]].values[0]

In [ ]:
artists_data=artists_df.tolist()

In [ ]:
cur.execute(artist_table_insert, artists_data)

### Process log_data to extract  time & users dimensional tables
- In this notebook, for the sake of demo, I'll perform ETL on a single log file and load it into each table to start.
- I'll use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`

In [ ]:
log_files=get_files("data/log_data")
log_files[0]

In [ ]:
df_logs=pd.read_json(log_files[0], lines=True)
df_logs.head(1)

#### Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime ( because the timestamp is in milliseconds 
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [ ]:
df_logs=df_logs[df_logs['page']=='NextSong']

In [ ]:
t=pd.to_datetime(df_logs['ts'],unit='ms')
time_data=[t,\
          t.dt.hour,\
          t.dt.day,\
          t.dt.week,\
          t.dt.month,\
          t.dt.year,\
          t.dt.dayofweek]
column_labels=['ts', 'hour', 'day', 'week', 'month', 'year', 'dayofweek']
time_df = pd.DataFrame(dict(zip(column_labels, time_data)))
time_df.head()

##### Insert Records row by row into Time table
I implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [ ]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, row.tolist())

#### users table

In [ ]:
users_df=df_logs[["userId", "firstName", "lastName", "gender", "level"]]

In [ ]:
for i, row in users_df.iterrows():
    cur.execute(user_table_insert, row.tolist())

#### songplays Table
##### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

##### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [ ]:
for i, row in df_logs.iterrows():
    cur.execute(song_select, (row.song, row.artist, row.length))
    results=cur.fetchone()
    
    if results:
        song_id, artist_id = results
    else :
        song_id, artist_id = None, None
    
    songplay_data=(pd.to_datetime(row.ts, unit='ms'),\
                   row.userId,\
                   song_id,\
                   artist_id,\
                   row.level,\
                   row.sessionId,\
                   row.location,\
                   row.userAgent)
    
    cur.execute(songplay_table_insert, songplay_data)                

### Close Connection to Sparkify Database

In [ ]:
conn.close()